In [ ]:
import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm

# Clasificacion de Generos

In [ ]:
#importamos los data del Data Analysis
data = pd.read_pickle("clean_data/track.pkl")
clean_features = pd.read_pickle("clean_data/features.pkl")


In [ ]:
#Las siguientes lineas no deberia copiarse ya que las estan  en la notebook de Data Analysis

In [ ]:
_features = ["chroma_cens", "chroma_cqt", "chroma_stft", "mfcc", "rmse", "spectral_bandwidth", "spectral_contrast", "spectral_rolloff", "tonnetz", "zcr"]
_fields = ["kurtosis", "mean", "std", "median", "max", "min"]

audio_features_df  = clean_features
audio_features_df.head()

In [ ]:
# Flatten features
tracks_with_extra_audio_features_df = pd.DataFrame(index=audio_features_df.index)
tracks_with_extra_audio_features_df

In [ ]:
## Initializing all needed columns with NaN
for index, row in audio_features_df.head(1).iterrows():
    print(index) # track id
    for feature in _features:
        for field in _fields:
            i = 0
            for k in row[feature][field]: # channel (if it is channel ?)                
                i += 1
                tracks_with_extra_audio_features_df[f'{feature}_{field[0:3]}_{i}'] = np.nan

In [ ]:
for index, row in audio_features_df.iterrows():
    for feature in _features:
        for field in _fields:
            i = 0
            for k in row[feature][field]: # channel (if it is channel ?)                
                i += 1
                tracks_with_extra_audio_features_df[f'{feature}_{field[0:3]}_{i}'] = k

In [ ]:
features= tracks_with_extra_audio_features_df


In [ ]:
#Librerias a importar

In [ ]:
from matplotlib import offsetbox
import joblib
#from PIL import Image
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.metrics import plot_confusion_matrix
#from imblearn.under_sampling import RandomUnderSampler

from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import auc, plot_roc_curve


## Generamos la Matriz de features 

In [ ]:
features_numericos=["duration", "acousticness","album_tracks", "danceability","energy","instrumentalness", "liveness",  "speechiness","tempo","valence"]

In [ ]:
#dropeo location porque tiene muchos nulos 
data.drop(labels=["location", "date_created","title", "album","artist"],axis=1, inplace= True)


In [ ]:
print("Media de las variables: ")
print(data.mean(axis=0))

print('\n')


print("Varianza de las variables: ")
print(data.var(axis=0))

In [ ]:
X = data.drop('genre_top', axis=1)
y = data['genre_top']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=7, stratify=y)

In [ ]:
X_train

In [ ]:
display(y_train.value_counts(normalize=True).round(2))

display(y_test.value_counts(normalize=True).round(2))

In [ ]:
#Scaling data

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_genre_fatures_sc = scaler.fit_transform(X_train)
X_train_sc_df = pd.DataFrame(train_genre_fatures_sc, columns = features_numericos)

scaler = StandardScaler()
train_genre_fatures_sc = scaler.fit_transform(X_test)
X_test_sc_df = pd.DataFrame(train_genre_fatures_sc, columns = features_numericos)


In [ ]:
print("Media de las variables: ")
print(X_train_sc_df.mean(axis=0))

print('\n')

# Observamos nuevamente la varianza de las variables: como normalizamos la varianza es 1
print("Varianza de las variables: ")
print(X_train_sc_df.var(axis=0))

In [ ]:
def custom_plot_confusion_matrix(confusion_matrix, genre_list=['Blues', 'Hip-Hop','Jazz', 'Pop', 'Rock']):
    df_cm = pd.DataFrame(confusion_matrix, 
                         index = genre_list, 
                         columns = genre_list)
    plt.figure(figsize = (14,9))
    sns.heatmap(df_cm, annot=True, fmt='g', cmap="Blues")

##  Modelo de NAIVE BAYES

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
gnb = GaussianNB()

gnb.fit(X_train_sc_df, y_train)

In [ ]:
Y_pred = gnb.predict(X_test_sc_df)

Y_pred

In [ ]:
round(accuracy_score(y_test, Y_pred), 2)

In [ ]:
print('Accuracy=', accuracy_score(y_test, Y_pred))
#print('Recall=', recall_score(Y_test, Y_pred))
#print('Precision=', precision_score(Y_test, Y_pred))

In [ ]:
sns.heatmap(confusion_matrix(y_test, Y_pred), annot=True, fmt='.0f')
plt.ylabel('Etiquetas reales')
plt.xlabel('Etiquetas predichas');


## Modelo de KNN

In [ ]:
# Importamos la clase KNeighborsClassifier de módulo neighbors
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn.fit(X_train_sc_df, y_train)

In [ ]:
y_pred = knn.predict(X_test_sc_df)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred).round(2)

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
kf = KFold(n_splits=12, shuffle=True, random_state=12)

scores_para_df = []

for i in range(1, 21):
    
    # En cada iteración, instanciamos el modelo con un hiperparámetro distinto
    model = KNeighborsClassifier(n_neighbors=i)
    
    # cross_val_scores nos devuelve un array de 5 resultados,
    # uno por cada partición que hizo automáticamente CV
    cv_scores = cross_val_score(model, X_train_sc_df, y_train, cv=kf)
    
    # Para cada valor de n_neighbours, creamos un diccionario con el valor
    # de n_neighbours y la media y el desvío de los scores
    dict_row_score = {'score_medio':np.mean(cv_scores),
                      'score_std':np.std(cv_scores), 'n_neighbors':i}
    
    # Guardamos cada uno en la lista de diccionarios
    scores_para_df.append(dict_row_score)

In [ ]:
df_scores = pd.DataFrame(scores_para_df)
df_scores.head()

In [ ]:
df_scores['limite_inferior'] = df_scores['score_medio'] - df_scores['score_std']
df_scores['limite_superior'] = df_scores['score_medio'] + df_scores['score_std']
df_scores.head()

In [ ]:
# Graficamos los resultados
plt.plot(df_scores['n_neighbors'], df_scores['limite_inferior'], color='r')
plt.plot(df_scores['n_neighbors'], df_scores['score_medio'], color='b')
plt.plot(df_scores['n_neighbors'], df_scores['limite_superior'], color='r');

In [ ]:
# Identificamos el score máximo
df_scores.loc[df_scores.score_medio == df_scores.score_medio.max()]

In [ ]:
# Asignamos el valor del k óptimo a una variable
best_k = df_scores_standard.loc[df_scores_standard.score_medio == df_scores_standard.score_medio.max(), 'n_neighbors'].values[0]
best_k

In [ ]:
# Elegimos el modelo óptimo de acuerdo a las pruebas de cross validation
model = KNeighborsClassifier(n_neighbors=best_k)

# Lo ajustamos sobre los datos de entrenamiento
model.fit(X_train, Y_train)

In [ ]:
#Evaluamos qué accuracy obtenemos en train
accuracy_score(Y_train, model.predict(X_train)).round(2)

In [ ]:
# Lo utilizamos para predecir en test
X_test = scaler.transform(X_test) # ¡Importantísimo estandarizar también los datos de test con las medias y desvíos aprendidos en train!
y_pred = model.predict(X_test)

In [ ]:
# Evaluamos el accuracy del modelo en test
accuracy_score(Y_test, y_pred).round(2)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

sns.set(rc={'figure.figsize':(30,30)})

# Graficamos la matriz de confusión
print(confusion_matrix(Y_test, y_pred))
plot_confusion_matrix(model,X_test, Y_test)

In [ ]:
sns.heatmap(confusion_matrix(Y_test, y_pred), annot=True, fmt='.0f')
plt.ylabel('Etiquetas reales')
plt.xlabel('Etiquetas predichas');

### Mejoras al modelo con GridSearch & Pipeline KNN

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline

folds=StratifiedKFold(n_splits=5,shuffle=True, random_state=42)

In [ ]:
pasos = [('scaler', StandardScaler()), ('knn', KNeighborsClassifier())]

In [ ]:
pipe_grid = Pipeline(pasos)

In [ ]:
param_grid = {'knn__n_neighbors':range(2,20,2),'knn__weights':['uniform','distance']}

In [ ]:
grid = GridSearchCV(pipe_grid, param_grid, cv=folds)
grid.fit(X_train_numerical_scaled, Y_train)

In [ ]:
grid.best_score_

In [ ]:
grid.best_estimator_

In [ ]:
accuracy_score(grid.best_estimator_.predict(X_test_numerical_scaled),Y_test)

### Modelamos unicamente teniendo encuenta los siguietne generos:
'Rock': 1500, 'Pop': 1105, 'Jazz': 440, 'Hip-Hop': 1048, 'Blues': 154

In [ ]:
# No pude hacer usar la libreria:  imblearn

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

sampler=RandomUnderSampler(sampling_strategy=undersampling_ratio)
Xu,yu=sampler.fit_resample(X_train,y_train)

yu.value_counts()

oversampling_ratio = dict(zip(list(y_train.unique()), [1500, 1500, 1200, 1500, 600]))
oversampling_ratio

categorical_mask=(X_train.dtypes=='category').values
sm=RandomOverSampler(sampling_strategy=oversampling_ratio)
X_train_rs,y_train_rs=sm.fit_resample(Xu,yu)

print(y_train_rs.shape)
print(y_train_rs.value_counts())

## GridSearch Stratified KFold - Random Forest Classifier

In [ ]:
 
model_instance = RandomForestClassifier()

gridSearch_params = {'n_estimators':[100, 500],
          'criterion': ['gini', 'entropy'],
          'max_depth': [None, 3, 7, 10, 15],
          'min_samples_split': [5, 7],
          'class_weight':[None, 'balanced']}


cv_Stratified_KFold = StratifiedKFold(n_splits=5, shuffle=True, random_state=77)

grid_search_CV_Stratified_KFold_model_RF = GridSearchCV(model_instance, gridSearch_params, n_jobs=-1, cv = cv_Stratified_KFold)
grid_search_CV_Stratified_KFold_model_RF.fit(X_train_rs, y_train_rs)

scores_Stratified_KFold = cross_val_score(model_instance, X_train_rs, y_train_rs, cv=cv_Stratified_KFold, n_jobs=-1)
mean_score_grid_search_CV_Stratified_KFold_model = scores_Stratified_KFold.mean()
std_score_grid_search_CV_Stratified_KFold_model = scores_Stratified_KFold.std()    

#score_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model_RF.best_score_
#params_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model_RF.best_params_

score_grid_search_CV_Stratified_KFold_model_test = grid_search_CV_Stratified_KFold_model_RF.score(X_test_sc_df, y_test)
predictions_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model_RF.predict(X_test_sc_df)
confusion_matrix_grid_search_CV_Stratified_KFold_model = metrics.confusion_matrix(y_test, predictions_grid_search_CV_Stratified_KFold_model)

In [ ]:
custom_plot_confusion_matrix(confusion_matrix(y_test, grid_search_CV_Stratified_KFold_model_RF.predict(X_test_sc_df)))

# XGBoost Para RandomForest Classifier

In [ ]:
from xgboost import XGBRFClassifier

In [ ]:
## Usando XG Boosting
model_instance = XGBRFClassifier(n_estimators=1000, subsample=0.9, colsample_bynode=0.2)

gridSearch_params = {'n_estimators':[1000],
                     'max_depth': [None, 3, 5]}


cv_Stratified_KFold = StratifiedKFold(n_splits=3, shuffle=True, random_state=77)

grid_search_CV_Stratified_KFold_model = GridSearchCV(model_instance, gridSearch_params, n_jobs=-1, cv = cv_Stratified_KFold)
grid_search_CV_Stratified_KFold_model.fit(X_train_rs, y_train_rs)

scores_Stratified_KFold = cross_val_score(model_instance, X_train_rs, y_train_rs, cv=cv_Stratified_KFold, n_jobs=-1)
mean_score_grid_search_CV_Stratified_KFold_model = scores_Stratified_KFold.mean()
std_score_grid_search_CV_Stratified_KFold_model = scores_Stratified_KFold.std()    

score_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model.best_score_
params_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model.best_params_

score_grid_search_CV_Stratified_KFold_model_test = grid_search_CV_Stratified_KFold_model.score(X_test_sc_df, y_test)
predictions_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model.predict(X_test_sc_df)
confusion_matrix_grid_search_CV_Stratified_KFold_model = metrics.confusion_matrix(y_test, predictions_grid_search_CV_Stratified_KFold_model)

In [ ]:
custom_plot_confusion_matrix(confusion_matrix(y_test, grid_search_CV_Stratified_KFold_model.predict(X_test_sc_df)))

y_test.value_counts()[0:5]

In [ ]:
params_grid_search_CV_Stratified_KFold_model

In [ ]:
mean_score_grid_search_CV_Stratified_KFold_model

In [ ]:
std_score_grid_search_CV_Stratified_KFold_model

In [ ]:
score_grid_search_CV_Stratified_KFold_model

In [ ]:
score_grid_search_CV_Stratified_KFold_model_test

In [ ]:
## Noisy data makes XG Random Forest Boost to perfom badly

In [ ]:
genres = ['Hip-Hop', 'Pop', 'Blues', 'Jazz']
echonest_track_ids = set(list(echonest.index))

original_genre_top_selected_genres_ids = ( set(list((tracks_genres_info[tracks_genres_info.genre_top == 'Hip-Hop']).index)) | 
                                           set(list((tracks_genres_info[tracks_genres_info.genre_top == 'Pop']).index))     |
                                           set(list((tracks_genres_info[tracks_genres_info.genre_top == 'Rock']).index))    | 
                                           set(list((tracks_genres_info[tracks_genres_info.genre_top == 'Blues']).index))   | 
                                           set(list((tracks_genres_info[tracks_genres_info.genre_top == 'Jazz']).index)))   

original_genre_top_selected_genres_ids = original_genre_top_selected_genres_ids &  echonest_track_ids # Only the ones that has Echonest features

In [ ]:
print("Original tracks: ", tracks_genres_info[tracks_genres_info.index.isin(original_genre_top_selected_genres_ids)].shape[0])

print(tracks_genres_info[tracks_genres_info.index.isin(original_genre_top_selected_genres_ids)].genre_top.value_counts())

In [ ]:
derived_genre_tracks_ids = {} 

for genre in genres:    
    derived_genre_tracks_ids[genre] = set(list(null_top_genre_tracks_df[null_top_genre_tracks_df.derived_genre_top == genre].index)) & (echonest_track_ids)

print("Derived Genre Tracks:")
for genre, ids in derived_genre_tracks_ids.items():
    print(f'\t{genre}: {len(ids)}')
    len(derived_genre_tracks_ids['Pop'])

flat_list_derived_genre_track_ids = sorted({x for v in derived_genre_tracks_ids.values() for x in v})

In [ ]:
genre_fatures_df = pd.concat([echonest.echonest.audio_features[echonest.index.isin(original_genre_top_selected_genres_ids)], tracks_info_with_derived_top_genre_df[tracks_info_with_derived_top_genre_df._genre_top.index.isin(original_genre_top_selected_genres_ids)]], 1)

derived_genre_fatures_df = pd.concat([echonest.echonest.audio_features[echonest.index.isin(flat_list_derived_genre_track_ids)], null_top_genre_tracks_df[null_top_genre_tracks_df.derived_genre_top.index.isin(flat_list_derived_genre_track_ids)]], 1)

In [ ]:
genre_fatures_df

derived_genre_fatures_df['genre_top'] = derived_genre_fatures_df.derived_genre_top

derived_genre_fatures_df.genre_top.value_counts()

In [1501]:
X = genre_fatures_df[full_features]
y = genre_fatures_df['genre_top']

In [1505]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7, stratify=y)

y_train.value_counts()

,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence
track_id,,,,,,,,
36730,0.002562,0.371206,0.948498,0.938766,0.099965,0.118478,120.170,0.052455
75751,0.175495,0.684260,0.682260,0.000092,0.126889,0.038587,89.989,0.332568
14184,0.728182,0.489313,0.301333,0.873091,0.161249,0.057750,75.044,0.614381
36746,0.000028,0.338520,0.988323,0.917932,0.069609,0.110313,143.774,0.089863
27607,0.575338,0.844497,0.454838,0.780293,0.076708,0.064396,130.051,0.352131
...,...,...,...,...,...,...,...,...
1890,0.171406,0.456081,0.680751,0.930712,0.081202,0.028629,107.080,0.496760
21521,0.179581,0.564068,0.743802,0.634654,0.117850,0.036937,128.056,0.813390
4108,0.968047,0.219570,0.457452,0.247248,0.247665,0.079178,164.333,0.229997


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_genre_fatures_sc = scaler.fit_transform(X_train)
X_train_sc_df = pd.DataFrame(train_genre_fatures_sc, columns = full_features)

scaler = StandardScaler()
train_genre_fatures_sc = scaler.fit_transform(X_test)
X_test_sc_df = pd.DataFrame(train_genre_fatures_sc, columns = full_features)

In [ ]:
undersampling_ratio = dict(zip(list(y_train.unique()), [1000, 637, 242, 46, 169]))
undersampling_ratio

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTENC

sampler=RandomUnderSampler(sampling_strategy=undersampling_ratio)
Xu,yu=sampler.fit_resample(X_train_sc_df,y_train)

yu.value_counts()

In [ ]:
# Extra Hip Hop for Training
extra_hip_hop_train_data = derived_genre_fatures_df[derived_genre_fatures_df.genre_top == 'Hip-Hop'][features + ['tempo']].sample(800 - 637)
Xu = Xu.append(extra_hip_hop_train_data.drop('genre_top', axis=1))
yu = yu.append(extra_hip_hop_train_data.genre_top.sample(800 - 637))

# Extra Pop for Training
extra_pop_train_data = derived_genre_fatures_df[derived_genre_fatures_df.genre_top == 'Pop'][features + ['tempo']].sample(600 - 242)
Xu = Xu.append(extra_pop_train_data.drop('genre_top', axis=1))
yu = yu.append(extra_pop_train_data.genre_top)

# Extra Jazz for Training
extra_jazz_train_data = derived_genre_fatures_df[derived_genre_fatures_df.genre_top == 'Jazz'][features + ['tempo']].sample(400 - 169)
Xu = Xu.append(extra_jazz_train_data.drop('genre_top', axis=1))
yu = yu.append(extra_jazz_train_data.genre_top)
               
# Extra Blues for Training
extra_blues_train_data = derived_genre_fatures_df[derived_genre_fatures_df.genre_top == 'Blues'][features + ['tempo']].sample(170 - 46)
Xu = Xu.append(extra_blues_train_data.drop('genre_top', axis=1))
yu = yu.append(extra_blues_train_data.genre_top)

print(Xu.shape[0], yu.shape[0])

oversampling_ratio = dict(zip(list(y_train.unique()), [1000, 900, 800, 600, 600]))
oversampling_ratio

In [ ]:
categorical_mask=(X_train.dtypes=='category').values
sm=RandomOverSampler(sampling_strategy=oversampling_ratio)
X_train_rs,y_train_rs=sm.fit_resample(Xu,yu)

print(y_train_rs.value_counts())    

In [ ]:
y_test.value_counts()

In [ ]:
clf_RF = RandomForestClassifier(max_depth=10, random_state=7, n_estimators=500)
clf_RF.fit(X_train_rs, y_train_rs)

In [ ]:
y_pred = clf_RF.predict(X_test_sc_df)
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average="macro"))
print(precision_score(y_test, y_pred, average="macro"))
print(recall_score(y_test, y_pred, average="macro"))    
custom_plot_confusion_matrix(confusion_matrix(y_test, clf_RF.predict(X_test_sc_df)))

In [ ]:
## Mejorando con GridSearch Stratified KFold   
model_instance = RandomForestClassifier()

gridSearch_params = {'n_estimators':[500, 1000],
          'criterion': ['gini', 'entropy'],
          'max_depth': [None, 3, 7, 10, 15],
          'min_samples_split': [5, 7],
          'class_weight':[None, 'balanced']}


cv_Stratified_KFold = StratifiedKFold(n_splits=5, shuffle=True, random_state=77)

grid_search_CV_Stratified_KFold_model_RF = GridSearchCV(model_instance, gridSearch_params, n_jobs=-1, cv = cv_Stratified_KFold)
grid_search_CV_Stratified_KFold_model_RF.fit(X_train_rs, y_train_rs)

scores_Stratified_KFold = cross_val_score(model_instance, X_train_rs, y_train_rs, cv=cv_Stratified_KFold, n_jobs=-1)
mean_score_grid_search_CV_Stratified_KFold_model = scores_Stratified_KFold.mean()
std_score_grid_search_CV_Stratified_KFold_model = scores_Stratified_KFold.std()    

#score_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model_RF.best_score_
#params_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model_RF.best_params_

score_grid_search_CV_Stratified_KFold_model_test = grid_search_CV_Stratified_KFold_model_RF.score(X_test_sc_df, y_test)
predictions_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model_RF.predict(X_test_sc_df)
confusion_matrix_grid_search_CV_Stratified_KFold_model = metrics.confusion_matrix(y_test, predictions_grid_search_CV_Stratified_KFold_model)

In [ ]:
y_pred = grid_search_CV_Stratified_KFold_model_RF.predict(X_test_sc_df)
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average="macro"))
print(precision_score(y_test, y_pred, average="macro"))
print(recall_score(y_test, y_pred, average="macro"))    
custom_plot_confusion_matrix(confusion_matrix(y_test, grid_search_CV_Stratified_KFold_model_RF.predict(X_test_sc_df)))

In [ ]:
## Usando XG Boosting
model_instance = XGBRFClassifier(n_estimators=1000, subsample=0.9, colsample_bynode=0.2)

gridSearch_params = {'n_estimators':[500, 1000, 1500],
                     'max_depth': [None, 3,4,5]}


cv_Stratified_KFold = StratifiedKFold(n_splits=4, shuffle=True, random_state=77)

grid_search_CV_Stratified_KFold_model = GridSearchCV(model_instance, gridSearch_params, n_jobs=-1, cv = cv_Stratified_KFold)
grid_search_CV_Stratified_KFold_model.fit(X_train_rs, y_train_rs)

scores_Stratified_KFold = cross_val_score(model_instance, X_train_rs, y_train_rs, cv=cv_Stratified_KFold, n_jobs=-1)
mean_score_grid_search_CV_Stratified_KFold_model = scores_Stratified_KFold.mean()
std_score_grid_search_CV_Stratified_KFold_model = scores_Stratified_KFold.std()    

score_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model.best_score_
params_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model.best_params_

score_grid_search_CV_Stratified_KFold_model_test = grid_search_CV_Stratified_KFold_model.score(X_test_sc_df, y_test)
predictions_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model.predict(X_test_sc_df)
confusion_matrix_grid_search_CV_Stratified_KFold_model = metrics.confusion_matrix(y_test, predictions_grid_search_CV_Stratified_KFold_model)

In [ ]:
y_pred = grid_search_CV_Stratified_KFold_model.predict(X_test_sc_df)
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average="macro"))
print(precision_score(y_test, y_pred, average="macro"))
print(recall_score(y_test, y_pred, average="macro"))    
custom_plot_confusion_matrix(confusion_matrix(y_test, y_pred))

In [ ]:
# Añadiendo información de anális de audio 
_features = ["chroma_cens", "chroma_cqt", "chroma_stft", "mfcc", "rmse", "spectral_bandwidth", "spectral_contrast", "spectral_rolloff", "tonnetz", "zcr"]
_fields = ["kurtosis", "mean", "std", "median", "max", "min"]

audio_features_df  = features_df[features_df.index.isin(list(genre_fatures_df.index) + list(derived_genre_fatures_df.index))][_features]
audio_features_df.head()

In [ ]:
# Flatten features
tracks_with_extra_audio_features_df = pd.DataFrame(index=audio_features_df.index)
tracks_with_extra_audio_features_df

In [ ]:
## Initializing all needed columns with NaN
for index, row in audio_features_df.head(1).iterrows():
    print(index) # track id
    for feature in _features:
        for field in _fields:
            i = 0
            for k in row[feature][field]: # channel (if it is channel ?)                
                i += 1
                tracks_with_extra_audio_features_df[f'{feature}_{field[0:3]}_{i}'] = np.nan

In [ ]:
for index, row in audio_features_df.iterrows():
    for feature in _features:
        for field in _fields:
            i = 0
            for k in row[feature][field]: # channel (if it is channel ?)                
                i += 1
                tracks_with_extra_audio_features_df[f'{feature}_{field[0:3]}_{i}'] = k

In [ ]:
tracks_with_extra_audio_features_df.head()

In [ ]:
extra_audio_genre_df = pd.concat([genre_fatures_df, tracks_with_extra_audio_features_df[tracks_with_extra_audio_features_df.index.isin(genre_fatures_df.index)]], 1)

extra_audio_derived_genre_df = pd.concat([derived_genre_fatures_df, tracks_with_extra_audio_features_df[tracks_with_extra_audio_features_df.index.isin(derived_genre_fatures_df.index)]], 1)

extra_audio_genre_df.head()


In [ ]:
X = extra_audio_genre_df.drop(['genre_top', '_genre_top', 'genres', 'genres_all', 'track_id'], axis=1)
y = extra_audio_genre_df['genre_top']

X.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7, stratify=y)

y_train.value_counts()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_genre_fatures_sc = scaler.fit_transform(X_train)
X_train_sc_df = pd.DataFrame(train_genre_fatures_sc, columns = X.columns)

scaler = StandardScaler()
train_genre_fatures_sc = scaler.fit_transform(X_test)
X_test_sc_df = pd.DataFrame(train_genre_fatures_sc, columns = X.columns)

In [ ]:
undersampling_ratio = dict(zip(list(y_train.unique()), [1000, 637, 242, 46, 169]))
undersampling_ratio

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTENC

sampler=RandomUnderSampler(sampling_strategy=undersampling_ratio)
Xu,yu=sampler.fit_resample(X_train_sc_df,y_train)

yu.value_counts()

In [ ]:
# Extra Hip Hop for Training

cols_to_drop = ['genre_top', 'genres', 'genres_all', 'track_id', 'genres_all_str', 'genres_str', 'derived_genre_top']

extra_hip_hop_train_data = extra_audio_derived_genre_df[extra_audio_derived_genre_df.genre_top == 'Hip-Hop'].sample(800 - 637)
Xu = Xu.append(extra_hip_hop_train_data.drop(cols_to_drop, axis=1))
yu = yu.append(extra_hip_hop_train_data.genre_top.sample(800 - 637))

# Extra Pop for Training
extra_pop_train_data = extra_audio_derived_genre_df[extra_audio_derived_genre_df.genre_top == 'Pop'].sample(600 - 242)
Xu = Xu.append(extra_pop_train_data.drop(cols_to_drop, axis=1))
yu = yu.append(extra_pop_train_data.genre_top)

# Extra Jazz for Training
extra_jazz_train_data = extra_audio_derived_genre_df[extra_audio_derived_genre_df.genre_top == 'Jazz'].sample(400 - 169)
Xu = Xu.append(extra_jazz_train_data.drop(cols_to_drop, axis=1))
yu = yu.append(extra_jazz_train_data.genre_top)
               
# Extra Blues for Training
extra_blues_train_data = extra_audio_derived_genre_df[extra_audio_derived_genre_df.genre_top == 'Blues'].sample(170 - 46)
Xu = Xu.append(extra_blues_train_data.drop(cols_to_drop, axis=1))
yu = yu.append(extra_blues_train_data.genre_top)

print(Xu.shape[0], yu.shape[0])
yu.value_counts()

In [ ]:
oversampling_ratio = dict(zip(list(y_train.unique()), [1000, 900, 800, 600, 600]))
oversampling_ratio

categorical_mask=(X_train_sc_df.dtypes=='category').values
sm=RandomOverSampler(sampling_strategy=oversampling_ratio)
X_train_rs,y_train_rs=sm.fit_resample(Xu,yu)

print(y_train_rs.value_counts())    
X_train_rs

In [ ]:
clf_RF_extra_audio = RandomForestClassifier(max_depth=10, random_state=7, n_estimators=500)
clf_RF_extra_audio.fit(X_train_rs, y_train_rs)

In [ ]:
# Para Todos los casos de prueba

y_pred = clf_RF_extra_audio.predict(X_test_sc_df)
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average="macro"))
print(precision_score(y_test, y_pred, average="macro"))
print(recall_score(y_test, y_pred, average="macro"))    
custom_plot_confusion_matrix(confusion_matrix(y_test, y_pred))

In [ ]:
import time

In [ ]:
## Mejorando con GridSearch Stratified KFold   
model_instance = RandomForestClassifier()

gridSearch_params = {'n_estimators':[500, 1000],
          'criterion': ['gini', 'entropy'],
          'max_depth': [None, 3, 7, 10, 15],
          'min_samples_split': [5, 7],
          'class_weight':[None, 'balanced']}

start = time.time()
cv_Stratified_KFold = StratifiedKFold(n_splits=5, shuffle=True, random_state=77)

grid_search_CV_Stratified_KFold_model_RF_extra_audio = GridSearchCV(model_instance, gridSearch_params, n_jobs=-1, cv = cv_Stratified_KFold)
grid_search_CV_Stratified_KFold_model_RF_extra_audio.fit(X_train_rs, y_train_rs)
stop = time.time()
print(f"Training time: {stop - start}s")

scores_Stratified_KFold = cross_val_score(model_instance, X_train_rs, y_train_rs, cv=cv_Stratified_KFold, n_jobs=-1)
mean_score_grid_search_CV_Stratified_KFold_model = scores_Stratified_KFold.mean()
std_score_grid_search_CV_Stratified_KFold_model = scores_Stratified_KFold.std()    

#score_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model_RF.best_score_
#params_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model_RF.best_params_

score_grid_search_CV_Stratified_KFold_model_test = grid_search_CV_Stratified_KFold_model_RF_extra_audio.score(X_test_sc_df, y_test)
predictions_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model_RF_extra_audio.predict(X_test_sc_df)
confusion_matrix_grid_search_CV_Stratified_KFold_model = metrics.confusion_matrix(y_test, predictions_grid_search_CV_Stratified_KFold_model)

In [ ]:
y_pred = grid_search_CV_Stratified_KFold_model_RF.predict(X_test_sc_df)
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average="macro"))
print(precision_score(y_test, y_pred, average="macro"))
print(recall_score(y_test, y_pred, average="macro"))    
custom_plot_confusion_matrix(confusion_matrix(y_test, y_pred))

In [ ]:
## Usando XG Boosting
model_instance = XGBRFClassifier(n_estimators=1000, subsample=0.9, colsample_bynode=0.2)

gridSearch_params = {'n_estimators':[500, 1000, 1500],
                     'max_depth': [None, 3,4,5]}

start = time.time()
cv_Stratified_KFold = StratifiedKFold(n_splits=4, shuffle=True, random_state=77)

grid_search_CV_Stratified_KFold_model_extra_audio = GridSearchCV(model_instance, gridSearch_params, n_jobs=-1, cv = cv_Stratified_KFold)
grid_search_CV_Stratified_KFold_model_extra_audio.fit(X_train_rs, y_train_rs)
stop = time.time()
print(f"Training time: {stop - start}s")

scores_Stratified_KFold = cross_val_score(model_instance, X_train_rs, y_train_rs, cv=cv_Stratified_KFold, n_jobs=-1)
mean_score_grid_search_CV_Stratified_KFold_model = scores_Stratified_KFold.mean()
std_score_grid_search_CV_Stratified_KFold_model = scores_Stratified_KFold.std()    

score_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model_extra_audio.best_score_
params_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model_extra_audio.best_params_

score_grid_search_CV_Stratified_KFold_model_test = grid_search_CV_Stratified_KFold_model_extra_audio.score(X_test_sc_df, y_test)
predictions_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model_extra_audio.predict(X_test_sc_df)
confusion_matrix_grid_search_CV_Stratified_KFold_model = metrics.confusion_matrix(y_test, predictions_grid_search_CV_Stratified_KFold_model)

In [ ]:
y_pred = grid_search_CV_Stratified_KFold_model.predict(X_test_sc_df)
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average="macro"))
print(precision_score(y_test, y_pred, average="macro"))
print(recall_score(y_test, y_pred, average="macro"))    
custom_plot_confusion_matrix(confusion_matrix(y_test, y_pred))

In [ ]:
# Aplicando Reducción de dimensionalidad (PCA)
from sklearn.decomposition import PCA

pca = PCA(n_components = 7, random_state=7)
X_train_pca = pca.fit_transform(X_train_sc_df)
X_test_pca = pca.transform(X_test_sc_df)

In [ ]:
explained_variance = pca.explained_variance_ratio_

In [ ]:
plt.plot(np.cumsum(explained_variance[0:10] * 100))
plt.xlabel('Número de Componentes')
plt.xlabel('Varianza  explicada')
plt.savefig('elbow_plot.png', dpi=80)

In [ ]:
# Undersampling 

undersampling_ratio = dict(zip(list(y_train.unique()), [1000, 637, 242, 46, 169]))
undersampling_ratio

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

sampler=RandomUnderSampler(sampling_strategy=undersampling_ratio)
Xu_pca,yu_pca=sampler.fit_resample(X_train_pca,y_train)

yu_pca.value_counts()

In [ ]:
oversampling_ratio = dict(zip(list(y_train.unique()), [1000, 800, 550, 400, 600]))
oversampling_ratio

In [ ]:
categorical_mask=(X_train_sc_df.dtypes=='category').values
sm=RandomOverSampler(sampling_strategy=oversampling_ratio)
X_train_rs_pca,y_train_rs_pca=sm.fit_resample(Xu_pca,yu_pca)

print(y_train_rs.value_counts())    

In [ ]:
y_test.value_counts()

X_test_pca.shape

In [ ]:
clf_RF_pca = RandomForestClassifier(max_depth=20, random_state=7, n_estimators=1000)
clf_RF_pca.fit(X_train_rs_pca, y_train_rs_pca)

In [ ]:
y_pred = clf_RF_pca.predict(X_test_pca)
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average="macro"))
print(precision_score(y_test, y_pred, average="macro"))
print(recall_score(y_test, y_pred, average="macro"))    
custom_plot_confusion_matrix(confusion_matrix(y_test, y_pred))

In [ ]:
## Usando XG Boosting
model_instance = XGBRFClassifier(subsample=0.9, colsample_bynode=0.2, eval_metric=["error", "logloss"])

gridSearch_params = {'n_estimators':[50, 100, 300],
                     'max_depth': [None, 3,4,5],
                     'learning_rate': [0.01, 0.1]}

start = time.time()
cv_Stratified_KFold = StratifiedKFold(n_splits=3, shuffle=True, random_state=77)

grid_search_CV_Stratified_KFold_model_extra_audio_pca = GridSearchCV(model_instance, gridSearch_params, n_jobs=-1, cv = cv_Stratified_KFold)
grid_search_CV_Stratified_KFold_model_extra_audio_pca.fit(X_train_rs_pca, y_train_rs_pca)
stop = time.time()
print(f"Training time: {stop - start}s")

scores_Stratified_KFold = cross_val_score(model_instance, X_train_rs, y_train_rs, cv=cv_Stratified_KFold, n_jobs=-1)
mean_score_grid_search_CV_Stratified_KFold_model = scores_Stratified_KFold.mean()
std_score_grid_search_CV_Stratified_KFold_model = scores_Stratified_KFold.std()    

score_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model_extra_audio_pca.best_score_
params_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model_extra_audio_pca.best_params_

score_grid_search_CV_Stratified_KFold_model_test = grid_search_CV_Stratified_KFold_model_extra_audio_pca.score(X_test_pca, y_test)
predictions_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model_extra_audio_pca.predict(X_test_pca)
confusion_matrix_grid_search_CV_Stratified_KFold_model = metrics.confusion_matrix(y_test, predictions_grid_search_CV_Stratified_KFold_model)

In [ ]:
y_pred = grid_search_CV_Stratified_KFold_model_extra_audio_pca.predict(X_test_pca)
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average="macro"))
print(precision_score(y_test, y_pred, average="macro"))
print(recall_score(y_test, y_pred, average="macro"))    
custom_plot_confusion_matrix(confusion_matrix(y_test, y_pred))

In [ ]:
## Usando XG Boosting - Cambiando Métrica

from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score

model_instance = XGBRFClassifier(eval_metric=["error", "logloss"])

gridSearch_params = {
                     'n_estimators': [5, 50, 100, 500, 1000],
                     'max_depth': [3, 4],
                     'learning_rate': [0.01, 0.1],
                     'min_child_weight': [1, 5, 10],
                     'gamma': [0.5, 1, 1.5, 2, 5],
                     'subsample': [0.6, 0.8, 1.0],
                     'colsample_bytree': [0.3, 0.6, 0.8],
                     'colsample_bynode': [0.2, 0.4, 0.6]
                    }

start = time.time()
cv_Stratified_KFold = StratifiedKFold(n_splits=3, shuffle=True, random_state=105)

grid_search_CV_Stratified_KFold_model_extra_audio_pca_ht = GridSearchCV(model_instance, gridSearch_params, n_jobs=-1, cv = cv_Stratified_KFold, verbose=3)
grid_search_CV_Stratified_KFold_model_extra_audio_pca_ht.fit(X_train_rs_pca, y_train_rs_pca)
stop = time.time()
print(f"Training time: {stop - start}s")

scores_Stratified_KFold = cross_val_score(model_instance, X_train_rs, y_train_rs, cv=cv_Stratified_KFold, n_jobs=-1)
mean_score_grid_search_CV_Stratified_KFold_model = scores_Stratified_KFold.mean()
std_score_grid_search_CV_Stratified_KFold_model = scores_Stratified_KFold.std()    

score_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model_extra_audio_pca_ht.best_score_
params_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model_extra_audio_pca_ht.best_params_

score_grid_search_CV_Stratified_KFold_model_test = grid_search_CV_Stratified_KFold_model_extra_audio_pca_ht.score(X_test_pca, y_test)
predictions_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model_extra_audio_pca_ht.predict(X_test_pca)
confusion_matrix_grid_search_CV_Stratified_KFold_model = metrics.confusion_matrix(y_test, predictions_grid_search_CV_Stratified_KFold_model)

In [ ]:
y_pred = grid_search_CV_Stratified_KFold_model_extra_audio_pca_ht.predict(X_test_pca)
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average="macro"))
print(precision_score(y_test, y_pred, average="macro"))
print(recall_score(y_test, y_pred, average="macro"))    
custom_plot_confusion_matrix(confusion_matrix(y_test, y_pred))